In [ ]:
# Package Setup
import Pkg;
# Pkg.update()

# Julia Packages
using Distributions
using LinearAlgebra
using Statistics
using PGFPlots

# Satellite Dynamics Packages
using SatelliteDynamics

# Load SatelliteTasking - Reclone to keep version current
Pkg.clone("..") # For some reason this doens't work with Pkg.add + PackageSpec. Why?
using SatelliteTasking
using SatelliteTasking.SatellitePlanning
using SatelliteTasking.Analysis

# Temporary for now
Pkg.add("JuMP")
Pkg.add("Gurobi")
using JuMP
using Gurobi

In [ ]:
# Configure simulation
epc0 = Epoch(2019, 1, 1, 0, 0, 0, tsys=:UTC) # Start of time span
epcf = Epoch(2019, 1, 2, 0, 0, 0, tsys=:UTC) # End of simulation time span

# Set Simulation Time Step
timestep = 1
dtmax    = 5

# Define Satellite Orbit
oe   = [R_EARTH + 500e3, 0, 90.0, 0, 0, 0]
eci0 = sOSCtoCART(oe, use_degrees=true)

# Numer of perturbed orbits to simulate
num_orbits = 3

# Set Perturbation Values 
pos_error = 5000 # Position knowledge error [m]
vel_error = 5    # Velocity knowledge error [m/s]
orb_mean  = zeros(Float64, 6)
orb_sdev  = vcat((pos_error/sqrt(3)*ones(Float64, 3))..., (vel_error/sqrt(3)*ones(Float64, 3))...)

# Simulate true and perturbed orbits
@time true_orbit, perturbed_orbits, eci_errors = simulate_orbits(num_orbits, epc0, epcf, eci0, orb_mean, orb_sdev, timestep=timestep, dtmax=dtmax);

In [ ]:
# Compute True and perturbed collects

# Load test images
@time images = load_images("../data/landsat_test.json", dwell_time=5.0);
# @time images = load_images("../data/landsat_test_150.json", dwell_time=5.0);
# @time images = load_images("../data/landsat_test_300.json", dwell_time=5.0);
# @time images = load_images("../data/landsat_test_600.json", dwell_time=5.0);
num_images = length(images)

# Compute true and perturbed opportunities
@time true_opportunities, perturbed_opportunities, mean_diff, sdev_diff, missing_opportunities = compute_perturbed_opportunities(true_orbit, perturbed_orbits, images, epc_step=3600);

# 
@time collects = compute_collects_by_number(true_opportunities, 10);

# Compute feasible collects
image_collects = group_image_collects(collects) # Group collects by image
num_feasible   = 0
for img in keys(image_collects)
    if length(image_collects[img]) > 0
        num_feasible += 1
    end
end
pct_feasible = num_feasible/num_images*100

println("$num_feasible out of $num_images images have collection opportunities.")

In [ ]:
println(sdev_diff[1, :])
println(sdev_diff[2, :])
println(sdev_diff[3, :])
println(missing_opportunities)

In [ ]:
# Plot Differences in Opportunities
Axis([
    Plots.Linear(1:24, sdev_diff[1, :], legendentry="Start of Window")
    Plots.Linear(1:24, sdev_diff[2, :], legendentry="End of Window")
    Plots.Linear(1:24, sdev_diff[3, :], legendentry="Window Duration")
    Plots.Linear(1:24, missing_opportunities, legendentry="Number of Missing Opportunities")
], width="10cm", height="10cm", legendPos="north west", xmin=0, xmax=24, ymin=0)

In [ ]:
# Graph planning
@time path, reward, image_list = sp_graph_policy(collects, Function[constraint_agility_single_axis], horizon=0.0, allow_repeats=false)

println("Total planning reward: $reward")
println("Number of images collected: $(length(image_list))/$num_images, $(length(image_list)/num_images*100)")
println("Number of feasible images collected: $(length(image_list))/$num_feasible, $(length(image_list)/num_feasible*100)")

In [7]:
# MILP planning
@time path, reward, image_list = sp_milp_policy(collects, Function[constraint_agility_single_axis], horizon=0.0, allow_repeats=false)

println("Total planning reward: $reward")
println("Number of images collected: $(length(image_list))/$num_images, $(length(image_list)/num_images*100)")
println("Number of feasible images collected: $(length(image_list))/$num_feasible, $(length(image_list)/num_feasible*100)")

Academic license - for non-commercial use only
Optimize a model with 52931 rows, 4635 columns and 110247 nonzeros
Variable types: 0 continuous, 4635 integer (4635 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Variable types: 0 continuous, 4635 integer (4635 binary)

Root relaxation: objective 1.250000e+02, 31 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                     125.0000000 4635.00000  3608%     -    0s
     0     0          -    0       125.00000  125.00000  0.00%     -    0s

Explored 0 nodes (89 simplex iterations) in 0.16 seconds
Thread count was 12 (of 12 available processors)

Solution count 1: 125 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.250000000000e+02, best bound 1.250000000000e+02

In [29]:
# MDP Planning

function mdp_compute_transitions(collects::Array{Collect, 1}, constraint_list; horizon=0::Real)

    # Sort Collects to ensure they are in time-asecnding order
    sort!(collects, by = x -> x.sow)

    # Compute possible transitions
    transitions = Dict{Collect, Array{Collect, 1}}()

    for start_collect in collects
        # List of valid edges/transitions for start_collect
        transitions[start_collect] = Array{Collect, 1}[]
        
        for end_collect in collects
            # Decide to insert edge
            if (start_collect == end_collect || 
                start_collect.image == end_collect.image ||
                end_collect.sow < start_collect.eow)
                # Skip insertion if same collect, image, or starts before the current
                # collection ends (no instantaneous manevers)
                continue
            elseif horizon > 0 && end_collect.sow > (start_collect.eow + horizon)
                # Since we know collects are sorted we can break building the
                # transition grarph if the distance from the next to the next start
                # is greater than the look-ahead horizon
                break
            else
                # Set transition valid by default
                valid_transition = true

                for constraint in constraint_list
                    # If not valid transition break early
                    if valid_transition == false
                        continue
                    end

                    # Use logical and to evaulate path feasibility on transitions
                    valid_transition = valid_transition && constraint(start_collect, end_collect)
                end

                if valid_transition
                    push!(transitions[start_collect], end_collect)
                end
            end
        end
    end

    states = sort!(collect(keys(transitions)), by = x -> x.sow)

    return states, transitions
end

function _mdp_update_state_values(Uk::Dict{Collect, <:Real}, Ukp::Dict{Collect, <:Real}, states::Array{Collect, 1}, transitions::Dict{Collect, Array{Collect, 1}})
    # Perform initial update
    for s in states

        r_s = nothing # State reward
        if length(transitions[s]) == 0
            r_s = 0.0
        end 

        for a in transitions[s]
            # Transition probability to action state is always 1.0
            r_a = a.image.reward + Uk[a]

            # Update optimal state value 
            if r_s == nothing || r_a > r_s
                r_s = r_a
            end

        Ukp[s] = r_s

        end
    end
    
end

function _mdp_extract_policy(U::Dict{Collect, <:Real}, states::Array{Collect, 1}, transitions::Dict{Collect, Array{Collect, 1}})
    policy = Dict{Collect, Union{Collect, Nothing}}()

    for s in states
        r_s = nothing
        if length(transitions[s]) == 0
            r_s = 0.0
        end

        a_max = nothing
        for a in transitions[s]
            # Transition probability to action state is always 1.0
            r_a = a.image.reward + U[a]

            # Update optimal state value 
            if r_s == nothing || r_a > r_s
                r_s   = r_a
                a_max = a
            end
        end

        policy[s] = a_max
    end

    return policy
end

function mdp_value_iteration(states::Array{Collect, 1}, transitions::Dict{Collect, Array{Collect, 1}}, eps=1e-6::Real)

    # Initialize Initial value function 
    Uk  = Dict{Collect, Float64}(s => 0.0 for s in states)
    Ukp = Dict{Collect, Float64}(s => 0.0 for s in states)
    k   = 1

    # Perofrm first round of value ierations
    _mdp_update_state_values(Uk, Ukp, states, transitions)

    # Iterate while not-converged
    resid = norm([Ukp[s] - Uk[s] for s in states])
    while resid > eps
        # Update values in UkP
        for (k,v) in Ukp
            Uk[k] = v
        end

        _mdp_update_state_values(Uk, Ukp, states, transitions)
        resid = norm([Ukp[s] - Uk[s] for s in states])
        k += 1

#         println("Finished iteration $k. Bellman residual: $resid")
    end

    println("Finished value iteration. Converged after $k iterations with Bellman residual: $resid")

    # Extract optimal policy from state values
    policy = _mdp_extract_policy(Ukp, states, transitions) 

    # Extract optimal path from policy
    max_s    = findmax(Ukp)[2]
    opt_path = Union{Collect,Nothing}[max_s]

    while policy[opt_path[end]] != nothing
        push!(opt_path, policy[opt_path[end]])
    end

    return Ukp, opt_path
end

function sp_mdp_policy(collects::Array{Collect, 1}, constraint_list; horizon=0::Real, allow_repeats=false::Bool, eps=1e-6::Real)
    # Compute States and Transitions
    states, transitions = mdp_compute_transitions(collects, constraint_list, horizon=horizon)

    # Solve graph for taskign policy
    Ukp, path = mdp_value_iteration(states, transitions, eps)

    reward     = findmax(Ukp)[2]
    image_list = [col.image for col in path]

    return path, reward, image_list
end

sp_mdp_policy (generic function with 1 method)

In [30]:
# MDP Planning

@time path, reward, image_list = sp_mdp_policy(collects, Function[constraint_agility_single_axis], horizon=0.0, allow_repeats=false)

println("Total planning reward: $reward")
println("Number of images collected: $(length(image_list))/$num_images, $(length(image_list)/num_images*100)")
println("Number of feasible images collected: $(length(image_list))/$num_feasible, $(length(image_list)/num_feasible*100)")

Finished value iteration. Converged after 419 iterations with Bellman residual: 0.0
148.756090 seconds (43.01 M allocations: 1.591 GiB, 0.51% gc time)
Total planning reward: Collect(Ptr: 4645994912, Orbit: 0, Image: 5019237488, Opportunity: 4611716000, Start: Epoch(2019-01-01T00:10:04.000Z), End: Epoch(2019-01-01T00:10:09.000Z))
Number of images collected: 419/150, 279.33333333333337
Number of feasible images collected: 419/125, 335.2
